In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sike
import numpy as np
import sike.analysis.post_processing as spp
import sike.analysis.plotting as sp
from pathlib import Path

In [2]:
# sike.setup(elements = ["H"], savedir = "..")

In [ ]:
num_x = 100
Te = np.linspace(1.0,100.0,num_x)
ne = 1e20*np.ones(num_x)
el = "O"
# vgrid = np.sqrt(2.0 * sike.utils.constants.EL_CHARGE * np.geomspace(2.5e-4,30e3,500) / sike.utils.constants.EL_MASS)
# c = sike.SIKERun(Te=Te, ne=ne, element=el, resolve_j = False, resolve_l = False, saha_boltzmann_init=True)
c = sike.SIKERun(Te=Te, ne=ne, element=el, resolve_j = False, resolve_l = False, saha_boltzmann_init=True, vgrid=sike.utils.constants.DEFAULT_VGRID, atomic_data_savedir="/Users/power8/Documents/01_code/03_sike/SIKE/sike_atomic_data")
# c = sike.SIKERun(Te=Te, ne=ne, element=el, resolve_j = False, resolve_l = False, saha_boltzmann_init=True, vgrid=sike.utils.constants.EXTENDED_VGRID)
# c = sike.SIKERun(Te=Te, ne=ne, element=el, resolve_j = False, resolve_l = False, saha_boltzmann_init=True, vgrid=vgrid)

In [ ]:
ds = c.evolve(1e3)
# ds = c.solve()

In [5]:
sp.plot_nz(ds,logy=False, logx=True, normalise=False)
sp.plot_Zavg(ds)
# sp.plot_Lz(ds,logy=True, logx=True, normalise=False)

In [42]:
sp.plot_Zavg(ds)